In [2]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import plotly.graph_objects as go

In [15]:
data = pd.read_json("..\Datasets\WDO@_M5.json")
data['time'] = pd.to_datetime(data['time'], unit='ms')

In [16]:
columns = ['time', 'open', 'high', 'low', 'close']
parsed_ds = data.loc[(data['time']>='2021-03-1 8:30:00')&(data['time']<='2021-03-30 18:30:00'), columns]
parsed_ds.reset_index(inplace=True, drop=True)
parsed_ds.head()

,time,open,high,low,close
0,2021-03-01 09:20:00,5603.5,5609.0,5603.0,5608.0
1,2021-03-01 09:25:00,5608.5,5611.5,5603.0,5603.5
2,2021-03-01 09:30:00,5604.0,5604.0,5590.5,5593.0
3,2021-03-01 09:35:00,5593.0,5602.0,5585.0,5592.0
4,2021-03-01 09:40:00,5592.0,5598.5,5583.0,5597.5


In [36]:
depth=backstep=3

In [37]:
highs = np.full(parsed_ds.shape[0],None)
lows = np.full(parsed_ds.shape[0],None)
last_high=parsed_ds.iloc[0]['high']
last_low=parsed_ds.iloc[0]['low']
for idx, row in parsed_ds.iterrows():
    if idx < depth:
        continue
    max = np.amax(parsed_ds[idx-depth:idx]['high'])
    min = np.amin(parsed_ds[idx-depth:idx]['low'])
    if max == parsed_ds.iloc[idx-1]['high'] and last_high!=max:
        highs[idx-1]=max
        last_high=max
    if min == parsed_ds.iloc[idx-1]['low'] and min!=last_low:
        lows[idx-1]=min
        last_low=min
    
    for b in range(2, backstep+1):
        if lows[idx-1]!=None and lows[idx-b]!=None and lows[idx-b]>lows[idx-1]:
            lows[idx-b]=None
        if highs[idx-1]!=None and highs[idx-b]!=None and highs[idx-b]<highs[idx-1]:
            highs[idx-b]=None


In [38]:
zz = np.full(parsed_ds.shape[0], None)

for idx in range(0, parsed_ds.shape[0]):
    if highs[idx] != None:
        zz[idx]=highs[idx]
        
        for last in reversed(range(idx)):
            if zz[last] != None:
                if zz[last] != highs[last]:
                    break
                if zz[last] == highs[last]:
                    if zz[last]>=zz[idx]:
                        zz[idx]=None
                        break
                    else:
                        zz[last]=None
                        break

    if lows[idx] != None:
        zz[idx]=lows[idx]

        for last in reversed(range(idx)):
            if zz[last] != None:
                if zz[last] != lows[last]:
                    break
                if zz[last] == lows[last]:
                    if zz[last]<=zz[idx]:
                        zz[idx]=None
                        break
                    else:
                        zz[last]=None
                        break
        
parsed_ds['zz']=zz
parsed_ds['lows']=lows
parsed_ds['highs']=highs

In [40]:
fig = go.Figure(data=[go.Scatter(
                x=parsed_ds.loc[(parsed_ds['zz'].notnull())]['time'],
                y=parsed_ds.loc[(parsed_ds['zz'].notnull())]['zz']
            )])
# fig = go.Figure(data=[go.Candlestick(x=parsed_ds['time'],
#                 open=parsed_ds['open'],
#                 high=parsed_ds['high'],
#                 low=parsed_ds['low'],
#                 close=parsed_ds['close'])])

# fig.add_trace(go.Scatter(
#     x=parsed_ds.loc[(parsed_ds['zz'].notnull())]['time'],
#     y=parsed_ds.loc[(parsed_ds['zz'].notnull())]['zz']
# ))


fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]), #hide weekends
        dict(bounds=[19, 9], pattern="hour"), #hide hours outside of 9am-6pm
    ]
)
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

93.0 32.5
